In [72]:
import pandas as pd
import statsmodels.api as sm
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
import pylab
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
import re
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
import numpy.random as nprnd
pd.set_option('display.max_columns', 500)
%matplotlib inline
import random





df = pd.read_csv('loanclean.csv' , encoding='latin-1')
df.head()


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,x
0,1,1,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,NaN,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0,0,5863.155187,5833.84,5000.00,863.16,0.0,0.0,0.00,Jan-15,171.62,NaN,Feb-18,0.0,NaN,1,Individual,860
1,2,2,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Charged Off,n,NaN,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0,0,1014.530000,1014.53,456.46,435.17,0.0,122.9,1.11,Apr-13,119.66,NaN,Oct-16,0.0,NaN,1,Individual,309
2,3,3,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,NaN,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0,0,3005.666844,3005.67,2400.00,605.67,0.0,0.0,0.00,Jun-14,649.91,NaN,Jun-17,0.0,NaN,1,Individual,606
3,6,6,5000,5000,5000.0,36 months,7.90%,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,Dec-11,Fully Paid,n,NaN,NaN,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0,Nov-04,3,NaN,NaN,9,0,7963,28.30%,12,f,0,0,5632.210000,5632.21,5000.00,632.21,0.0,0.0,0.00,Jan-15,161.03,NaN,Feb-17,0.0,NaN,1,Individual,852
4,8,8,3000,3000,3000.0,36 months,18.64%,109.43,E,E1,MKC Accounting,9 years,RENT,48000.0,Source Verified,Dec-11,Fully Paid,n,NaN,Borrower added on 12/16/11 > Downpayment for...,car,Car Downpayment,900xx,CA,5.35,0,Jan-07,2,NaN,NaN,4,0,8221,87.50%,4,f,0,0,3939.135294,3939.14,3000.00,939.14,0.0,0.0,0.00,Jan-15,111.34,NaN,Dec-14,0.0,NaN,1,Individual,900


In [73]:
import pandas as pd
import statsmodels.api as sm
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
import pylab
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
import re
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
import numpy.random as nprnd
pd.set_option('display.max_columns', 500)
%matplotlib inline
import random

In [74]:
cols = [0,1,9,10,17,18,19,21,22,23,26,28,29,35,36,37,47,48,49,50,51,52,53] # dropped columns with text or mostly missing values
df2 = df.drop(df.columns[cols],1)
df3 = df2.drop(['last_pymnt_d', 'last_pymnt_amnt', 'collection_recovery_fee', 'recoveries', 'total_rec_late_fee', 'total_rec_int', 'total_rec_prncp', 'total_pymnt_inv', 'total_pymnt','issue_d'],1)

df3.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000,5000,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,27.65,0,1,3,0,13648,83.70%,9
1,2500,2500,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,1.00,0,5,3,0,1687,9.40%,4
2,2400,2400,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,8.72,0,2,2,0,2956,98.50%,10
3,5000,5000,5000.0,36 months,7.90%,156.46,A,3 years,RENT,36000.0,Source Verified,Fully Paid,wedding,11.20,0,3,9,0,7963,28.30%,12
4,3000,3000,3000.0,36 months,18.64%,109.43,E,9 years,RENT,48000.0,Source Verified,Fully Paid,car,5.35,0,2,4,0,8221,87.50%,4


In [75]:
df3['int_rate'] = df3['int_rate'].replace({'\%':''}, regex = True)
df3['revol_util'] = df3['revol_util'].replace({'\%':''}, regex = True)

In [76]:
df3_term=pd.get_dummies(df3['term'])
df3_term.head()

,36 months,60 months
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0


In [77]:
df3_grade=pd.get_dummies(df3['grade'])
df3_grade.head()

,A,B,C,D,E,F,G
0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0
2,0,0,1,0,0,0,0
3,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0


In [78]:
#df3 = df3.replace(['MORTGAGE', 'OTHER', 'NONE', 'RENT'], 'Do not own')
#df3 = df3.replace('OWN', 'Own')
df3_home_ownership=pd.get_dummies(df3['home_ownership'])
df3_home_ownership.head()

,MORTGAGE,NONE,OTHER,OWN,RENT
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [79]:
df3_verification_status=pd.get_dummies(df3['verification_status'])
df3_verification_status.head()

,Not Verified,Source Verified,Verified
0,0,0,1
1,0,1,0
2,1,0,0
3,0,1,0
4,0,1,0


In [80]:
#df3 = df3.replace(['4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years'], '4 years and above')
#df3 = df3.replace(['1 year', '2 years'], '1 to 2 years')
#df3 = df3.replace('n/a', 'unemployed')
df3_emp_length=pd.get_dummies(df3['emp_length'])
df3_emp_length.head()

,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a
0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0


In [81]:
#df3['issue_d'] = df3['issue_d'].str[-2:]
#df3_issue_d=pd.get_dummies(df3['issue_d'])
#df3_issue_d.columns = ['issue_d_2007', 'issue_d_2008', 'issue_d_2009', 'issue_d_2010', 'issue_d_2011']
#df3_issue_d.head()

In [82]:
df3 = df3.replace(['Charged Off', 'Does not meet the credit policy. Status:Charged Off'], '1')
df3 = df3.replace(['Fully Paid','Does not meet the credit policy. Status:Fully Paid'], '0')
df3.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24000.0,Verified,0,credit_card,27.65,0,1,3,0,13648,83.70,9
1,2500,2500,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30000.0,Source Verified,1,car,1.00,0,5,3,0,1687,9.40,4
2,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12252.0,Not Verified,0,small_business,8.72,0,2,2,0,2956,98.50,10
3,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,36000.0,Source Verified,0,wedding,11.20,0,3,9,0,7963,28.30,12
4,3000,3000,3000.0,36 months,18.64,109.43,E,9 years,RENT,48000.0,Source Verified,0,car,5.35,0,2,4,0,8221,87.50,4


In [83]:
#df3 = df3.replace(['car', 'small_business','educational','home_improvement','house','major_purchase','medical','moving','renewable_energy','vacation','wedding','credit_card'], 'other')
df3_purpose=pd.get_dummies(df3['purpose'])
df3_purpose.head()

,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [107]:
df3['annual_inc']= df3['annual_inc']/1000
df3.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24.000,Verified,0,credit_card,27.65,0,1,3,0,13648,83.70,9
1,2500,2500,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30.000,Source Verified,1,car,1.00,0,5,3,0,1687,9.40,4
2,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12.252,Not Verified,0,small_business,8.72,0,2,2,0,2956,98.50,10
3,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,36.000,Source Verified,0,wedding,11.20,0,3,9,0,7963,28.30,12
4,3000,3000,3000.0,36 months,18.64,109.43,E,9 years,RENT,48.000,Source Verified,0,car,5.35,0,2,4,0,8221,87.50,4


In [108]:
df3_clean = pd.concat([df3,df3_verification_status, df3_term, df3_grade, df3_home_ownership, df3_emp_length, df3_purpose],axis=1)
df3_clean.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,Not Verified,Source Verified,Verified,36 months,60 months,A,B,C,D,E,F,G,MORTGAGE,NONE,OTHER,OWN,RENT,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,5000,5000,4975.0,36 months,10.65,162.87,B,10+ years,RENT,24.000,Verified,0,credit_card,27.65,0,1,3,0,13648,83.70,9,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2500,2500,2500.0,60 months,15.27,59.83,C,< 1 year,RENT,30.000,Source Verified,1,car,1.00,0,5,3,0,1687,9.40,4,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2400,2400,2400.0,36 months,15.96,84.33,C,10+ years,RENT,12.252,Not Verified,0,small_business,8.72,0,2,2,0,2956,98.50,10,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,5000,5000,5000.0,36 months,7.90,156.46,A,3 years,RENT,36.000,Source Verified,0,wedding,11.20,0,3,9,0,7963,28.30,12,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,3000,3000,3000.0,36 months,18.64,109.43,E,9 years,RENT,48.000,Source Verified,0,car,5.35,0,2,4,0,8221,87.50,4,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [111]:
df3_perf = df3_clean.drop(['term','grade','emp_length','home_ownership','verification_status','purpose'],1)
df3_perf.head()
df3_perf = df3_perf.dropna(axis=0, how='any')
df3_perf.head()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,Not Verified,Source Verified,Verified,36 months,60 months,A,B,C,D,E,F,G,MORTGAGE,NONE,OTHER,OWN,RENT,1 year,10+ years,2 years,3 years,4 years,5 years,6 years,7 years,8 years,9 years,< 1 year,n/a,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,wedding
0,5000,5000,4975.0,10.65,162.87,24.000,0,27.65,0,1,3,0,13648,83.70,9,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2500,2500,2500.0,15.27,59.83,30.000,1,1.00,0,5,3,0,1687,9.40,4,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2400,2400,2400.0,15.96,84.33,12.252,0,8.72,0,2,2,0,2956,98.50,10,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,5000,5000,5000.0,7.90,156.46,36.000,0,11.20,0,3,9,0,7963,28.30,12,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,3000,3000,3000.0,18.64,109.43,48.000,0,5.35,0,2,4,0,8221,87.50,4,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [112]:
df3_perf.to_csv('loan_cleaned_data.csv', sep=',', index = False)